In [281]:
import gurobipy as gp
from gurobipy import GRB
from math import floor
import pandas as pd

# Useful Variable Definitions

# Num 5 min interval in an hour
hrIntervals = 12 
# Num of decision variables in model
numDecisionVars = 7

# Create Parameters

# Default Paramters (Don't Change)

# Maximum discharge rate (MW)
maxDischargeRate = 2
# Maximum charge rate (MW)
maxChargeRate = -2
# Storage capacity of battery (MWh)
bStorage = 2
# Round Trip Efficiency of battery (unitless)
roundTripEfficiency = 0.81


# Input Paramters 

# Storage capacity of battery at time zero (MWh)
bStorage0 = 0

# Price predictions for energy ($/MWh)
df = pd.read_csv('prices.csv')
prices = df['Prices'].values.tolist()

try:

    # Create a new model
    m = gp.Model("batOptimser")

    # Create variables

    dispatchGen = {}
    dischargeBattery = {}
    chargeBattery = {}
    energyInStorage = {}
    dispatchCost = {}
    dispatchRevenue = {}
    dispatchProfit = {}
    
    for i, p in enumerate(prices):
        # Decision Variables
        dispatchGen[i] = m.addVar(lb=-GRB.INFINITY,name="dispatchGen")
        dischargeBattery[i] = m.addVar(lb=0,ub=maxDischargeRate,name="dischargeBattery")
        chargeBattery[i] = m.addVar(lb=maxChargeRate,ub=0,name="chargeBattery")
        energyInStorage[i] = m.addVar(lb=0,ub=bStorage,name="energyInStorage")
        
        dispatchCost[i] = m.addVar(lb=-GRB.INFINITY,name="dispatchCost")
        dispatchRevenue[i] = m.addVar(lb=-GRB.INFINITY,name="dispatchRevenue")
        dispatchProfit[i] = m.addVar(lb=-GRB.INFINITY,name="dispatchProfit",obj=-1) # Objective Function (negative to maximize)
        
        # Constraints
        if (i > 0):
            preserveEnergyInStorage = m.addConstr(energyInStorage[i] == (energyInStorage[i-1]) - (roundTripEfficiency*chargeBattery[i-1]/12) - (dischargeBattery[i-1]/12))
        
        batteryDispatchDefinition = m.addConstr(dispatchGen[i] == dischargeBattery[i] + chargeBattery[i])
        
        dispatchCostDefinition = m.addConstr(dispatchCost[i] == -1*chargeBattery[i]*p/12)
        dispatchRevenueDefinition = m.addConstr(dispatchRevenue[i] == dischargeBattery[i]*p/12)
        dispatchProfitDefinition = m.addConstr(dispatchProfit[i] == dispatchRevenue[i] - dispatchCost[i])
        
     
    # More Constraints
    EnergyInStorageCapacity0 = m.addConstr(energyInStorage[0] == bStorage0)
    
    # Preserve Energy In Storage Definition for final price (misses it in for loop)
    energyInStorage[len(prices)] = m.addVar(lb=0,ub=bStorage,name="energyInStorage")
    preserveEnergyInStorage = m.addConstr(energyInStorage[len(prices)] == (energyInStorage[len(prices)-1]) - (roundTripEfficiency*chargeBattery[len(prices)-1]/12) - (dischargeBattery[len(prices)-1]/12))

    # Optimize model
    m.optimize()

    
    # Stats + Debugging
    
    profit = 0
    cost = 0
    charges = 0
    chargeMW = 0
    discharges = 0
    dischargeMW = 0
    waiting = 0
    
    for i,v in enumerate(m.getVars()):
        if (v.varName == "dispatchGen"):
            if (v.x > 0):
                action = "DISCHARGE"
                profit += v.x*prices[floor(i/numDecisionVars)]/hrIntervals
                discharges += 1
                dischargeMW += v.x
            elif (v.x < 0):
                action = "CHARGE"
                profit += v.x*prices[floor(i/numDecisionVars)]/hrIntervals
                cost += -v.x*prices[floor(i/numDecisionVars)]/hrIntervals
                charges += 1
                chargeMW += -v.x 
            else:
                action = "DO NOTHING"
                waiting += 1
            
            # DEBUGGING FOR DECISION VARIABLES -> Iteration number, price and action assigned
            print("\nPrice Interval %g, Price: " % ((i/numDecisionVars)+1), (prices[floor(i/numDecisionVars)]))
            print("Action = %s" % (action))
        
        # DEBUGGING FOR DECISIONS VARIABLES -> Variable names and values assigned
        print('%s %g' % (v.varName, v.x))

    # OPTIMAL SOLUTION STATS    
    
    print('\nModel objective value: %.4g' % m.objVal)
    print("Actual Profit = $%.4g -> %.2g%% Profit" % (profit, 100*profit/cost))
    print("Charged %.4g MW over %d charges (Lost %.4g MW due to battery inefficiency)" % (chargeMW, charges, chargeMW-(roundTripEfficiency*chargeMW)))
    print("Discharges %.4g MW over %s discharges" % (dischargeMW, discharges))
    print("Did nothing during %d time intervals" % (waiting))
    print("\nBattery action this time interval: %s" % (action))
    

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1441 rows, 2017 columns and 4033 nonzeros
Model fingerprint: 0xd7cae7d2
Coefficient statistics:
  Matrix range     [7e-02, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+00, 2e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1154 rows and 1157 columns
Presolve time: 0.00s
Presolved: 287 rows, 860 columns, 1146 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -9.3747527e+02   6.246121e+03   0.000000e+00      0s
      26   -1.4730773e+01   0.000000e+00   0.000000e+00      0s

Solved in 26 iterations and 0.01 seconds
Optimal objective -1.473077254e+01

Price Interval 1, Price:  27.43703
Action = DO NOTHING
dispatchGen 0
dischargeBattery 0
chargeBattery 0
energyInStorage 0
dispatchCost 0
dispatchRevenue 0
dispatchProfit 0

Price Interval 2, Price:  24.3
Action = DO NOTHING
dispatchGen 0
dischargeBattery 0
chargeBattery 0
energyInStorage 0
dispatchCost 0
d

Price Interval 249, Price:  13.670860000000001
Action = DO NOTHING
dispatchGen 0
dischargeBattery 0
chargeBattery 0
energyInStorage 0
dispatchCost 0
dispatchRevenue 0
dispatchProfit 0

Price Interval 250, Price:  12.009839999999999
Action = CHARGE
dispatchGen -2
dischargeBattery 0
chargeBattery -2
energyInStorage 0
dispatchCost 2.00164
dispatchRevenue 0
dispatchProfit -2.00164

Price Interval 251, Price:  14.29
Action = DO NOTHING
dispatchGen 0
dischargeBattery 0
chargeBattery 0
energyInStorage 0.135
dispatchCost 0
dispatchRevenue 0
dispatchProfit 0

Price Interval 252, Price:  15.54486
Action = DISCHARGE
dispatchGen 0.86
dischargeBattery 0.86
chargeBattery 0
energyInStorage 0.135
dispatchCost 0
dispatchRevenue 1.11405
dispatchProfit 1.11405

Price Interval 253, Price:  14.409
Action = DO NOTHING
dispatchGen 0
dischargeBattery 0
chargeBattery 0
energyInStorage 0.0633333
dispatchCost 0
dispatchRevenue 0
dispatchProfit 0

Price Interval 254, Price:  13.59504
Action = DO NOTHING
dispatchG